<a href="https://colab.research.google.com/github/KimNikita/machine-learning-practice/blob/main/Anime%20rating%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# Постановка задачи

## Задача восстановления регрессии: предсказание ожидаемого конечного пользовательского рейтинга аниме по его характеристикам
## Область применения: упрощение выбора аниме для просмотра из числа вышедших в текущем сезоне года. Проблема состоит в том, что процент действительно "хороших" аниме среди вышедших довольно низок. При этом, чтобы отделить "хорошие" от "плохих", необходимо вручную просмотреть как минимум половину серий, что в худшем случае обернется потерей времени и настроения. А ориентироваться на пользовательский рейтинг первые полгода - бессмысленно ввиду разных причин (конкретика опускается).


# Чтение данных

In [2]:
url = "https://raw.githubusercontent.com/KimNikita/machine-learning-practice/main/anime-dataset-2023.csv"
data_raw = pd.read_csv(url)

In [3]:
data_raw.shape

(24905, 24)

In [ ]:
data_raw.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


#Удаление ненужных столбцов

In [100]:
data = data_raw.drop(data_raw.columns[[0, 1, 3, 6, 9, 11, 12, 13, 14, 16, 18, 19, 20, 21, 22, 23]], axis=1)
data.head()

,English name,Score,Genres,Type,Episodes,Premiered,Source,Rating
0,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,spring 1998,Original,R - 17+ (violence & profanity)
1,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi",Movie,1.0,UNKNOWN,Original,R - 17+ (violence & profanity)
2,Trigun,8.22,"Action, Adventure, Sci-Fi",TV,26.0,spring 1998,Manga,PG-13 - Teens 13 or older
3,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,summer 2002,Original,PG-13 - Teens 13 or older
4,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,fall 2004,Manga,PG - Children


#Удаление ненужных строк (фильтрация)

In [101]:
data = data.loc[data['Score'] != 'UNKNOWN']
data = data.loc[data['Episodes'] != '1.0']
data = data.loc[data['Type'].isin(['ONA', 'TV', 'UNKNOWN'])]
data = data.loc[data['Source'].isin(['Original', 'Manga' 'Light novel', '4-koma manga', 'Visual novel', 'Other', 'Novel', 'Game', 'Book', 'Web manga', 'Mixed media', 'Card game', 'Web novel', 'UNKNOWN'])]

## Удаление "плохих" жанров

In [102]:
cid = data.columns.get_loc('Genres')
rows_to_drop=[]
for i in range(data.shape[0]):
  cur = data.iat[i, cid]
  if cur.find('Hentai') != -1 or cur.find('Erotica') != -1:
    rows_to_drop.append(i)

data.reset_index(inplace=True)
data.drop(rows_to_drop, inplace=True)
data.drop('index', inplace=True, axis=1)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)

## Удаление сиквелов \\ приквелов

In [103]:
def mycmp(name):
  return len(name[0])

names=[]
i=0
for name in data['English name']:
  names.append((name, i))
  i+=1
names = sorted(names, key=mycmp)

rows_to_drop=[]
for index, name in enumerate(names):
  spaces = name[0].count(' ')
  for i in range(index+1, data.shape[0]):
    if names[i][0].count(' ') > spaces and names[i][0].startswith(name[0]):
      rows_to_drop.append(names[i][1])

data.drop(rows_to_drop, inplace=True)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)
data.drop('English name', inplace=True, axis=1)

#Преобразование данных

## Столбец Genres

In [104]:
unique_genres={}
cid = data.columns.get_loc('Genres')
for i in range(data.shape[0]):
  cur = data.iat[i, cid]
  genres = cur.split(', ')
  for genre in genres:
    if genre in unique_genres:
      unique_genres[genre]+=1
    else:
      unique_genres[genre]=0
unique_genres = sorted(unique_genres.items(), key=lambda item: item[1], reverse=True)
print(data.shape)
unique_genres

(2635, 7)


[('Comedy', 983),
 ('Action', 891),
 ('Fantasy', 733),
 ('Adventure', 608),
 ('Sci-Fi', 590),
 ('Drama', 417),
 ('Slice of Life', 326),
 ('Romance', 317),
 ('Supernatural', 200),
 ('Mystery', 188),
 ('UNKNOWN', 130),
 ('Ecchi', 97),
 ('Sports', 83),
 ('Horror', 69),
 ('Suspense', 46),
 ('Award Winning', 32),
 ('Gourmet', 32),
 ('Boys Love', 21),
 ('Girls Love', 15),
 ('Avant Garde', 11)]

In [105]:
usable_genres=[]
for genre in unique_genres:
  if float(genre[1]) > data.shape[0]*0.05:
    usable_genres.append(genre[0])
    data.insert(cid+1, 'Genre ' + genre[0], 0)

usable_genres

['Comedy',
 'Action',
 'Fantasy',
 'Adventure',
 'Sci-Fi',
 'Drama',
 'Slice of Life',
 'Romance',
 'Supernatural',
 'Mystery']

In [106]:
rows_to_drop=[]
for i in range(data.shape[0]):
  cur = data.iat[i, cid]
  genres = cur.split(', ')
  f=0
  for genre in genres:
    if genre not in usable_genres:
      f=1
      rows_to_drop.append(i)
      break
  if f==0:
    for genre in genres:
      data.iat[i, data.columns.get_loc('Genre ' + genre)]=1

data.drop(rows_to_drop, inplace=True)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)
data.drop('Genres', inplace=True, axis=1)
data.shape

(2117, 16)

## Столбец Episodes

In [107]:
cid = data.columns.get_loc('Episodes')
for i in range(data.shape[0]):
  cur = data.iat[i, cid]
  if cur != 'UNKNOWN':
    cur = float(cur)
    if cur <= 18:
      data.iat[i, cid] = 'half-season'
    elif 18 < cur <= 36:
      data.iat[i, cid] = 'season'
    elif 36 < cur <= 60:
      data.iat[i, cid] = 'double-season'
    elif 60 < cur:
      data.iat[i, cid] = 'no-season'

## Столбец Premiered

In [108]:
cid = data.columns.get_loc('Premiered')
for i in range(data.shape[0]):
  cur = data.iat[i, cid]
  if cur.startswith('spring'):
    data.iat[i, cid] = 'spring'
  elif cur.startswith('summer'):
    data.iat[i, cid] = 'summer'
  elif cur.startswith('fall'):
    data.iat[i, cid] = 'fall'
  elif cur.startswith('winter'):
    data.iat[i, cid] = 'winter'


# Борьба с выбросами \\ пропущенными значениями (UNKNOWN)

In [109]:
data['Score'] = data['Score'].astype('float')
rows_to_drop = data[(data['Score'] < data['Score'].quantile(0.005)) | (data['Score'] > data['Score'].quantile(0.995))].index
data.drop(rows_to_drop, inplace=True)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)
data.shape


(2095, 16)

In [110]:
print(data['Type'].value_counts())
print(data['Episodes'].value_counts())
print(data['Premiered'].value_counts())
print(data['Source'].value_counts())
print(data['Rating'].value_counts())

TV     1511
ONA     584
Name: Type, dtype: int64
half-season      1196
season            444
double-season     357
no-season          74
UNKNOWN            24
Name: Episodes, dtype: int64
UNKNOWN    591
spring     492
fall       423
winter     332
summer     257
Name: Premiered, dtype: int64
Original        940
Game            267
Novel           234
Web manga       151
Other           147
4-koma manga    123
Visual novel     97
Mixed media      38
Web novel        34
Book             32
Card game        32
Name: Source, dtype: int64
PG-13 - Teens 13 or older         1310
G - All Ages                       425
PG - Children                      169
R - 17+ (violence & profanity)     140
R+ - Mild Nudity                    35
UNKNOWN                             16
Name: Rating, dtype: int64


In [111]:
data=data.loc[data['Episodes'].isin(['half-season', 'season', 'double-season'])]
data=data.loc[data['Source'].isin(['Original', 'Game', 'Novel', 'Web manga', 'Other', '4-koma manga', 'Visual novel'])]
data=data.loc[data['Rating'].isin(['PG-13 - Teens 13 or older', 'G - All Ages', 'PG - Children', 'R - 17+ (violence & profanity)'])]
data.shape

(1827, 16)

In [112]:
print(data['Premiered'].value_counts())

UNKNOWN    504
spring     442
fall       357
winter     293
summer     231
Name: Premiered, dtype: int64


# Преобразование типов

In [113]:
data['Type'] = data['Type'].astype('category')
data['Episodes'] = data['Episodes'].astype('category')
data['Premiered'] = data['Premiered'].astype('category')
data['Source'] = data['Source'].astype('category')
data['Rating'] = data['Rating'].astype('category')

# Визуализация

# Нормализация

In [114]:
data = pd.concat((data, pd.get_dummies(data['Type'])), axis=1)
data = pd.concat((data, pd.get_dummies(data['Episodes'])), axis=1)
data = pd.concat((data, pd.get_dummies(data['Premiered'])), axis=1)
data = pd.concat((data, pd.get_dummies(data['Source'])), axis=1)
data = pd.concat((data, pd.get_dummies(data['Rating'])), axis=1)

data.drop('Type', inplace=True, axis=1)
data.drop('Episodes', inplace=True, axis=1)
data.drop('Premiered', inplace=True, axis=1)
data.drop('Source', inplace=True, axis=1)
data.drop('Rating', inplace=True, axis=1)
data.shape

(1827, 32)

# Machine learning

# TODO
Визуализировать данные и вычислить основные характеристики (среднее, разброс, корреляционную матрицу и т.д.). Интерпретировать.
## Добавить текстовый признак "сюжет"?
\
1 Разбить данные на обучающую и тестовую выборки\
2 Запустить классификатор (регрессию) ближайших соседей или другой (аргументировать свой выбор)\
3 Подобрать оптимальное значение к-ва ближайших соседей (или другого релевантного гиперпараметра)\
4 Вычислить ошибки на обучающей и тестовой выборках. Сделать выводы\
5 По желанию: запустить другие классификаторы. Сравнить результаты\
6 По желанию: как-то побороться с несбалансированностью классов (если она есть)\
7 По желанию: исключить коррелированные переменные (объяснить зачем)\
8 Сделать общие выводы

